In [101]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
from datetime import datetime as dt

from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools
import warnings
warnings.filterwarnings("ignore")

In [112]:
fund_df = pd.read_csv("sp500_fundamental_199601_202502.csv")

In [103]:
df_daily_price = pd.read_csv("sp500_price_199601_202502.csv")

In [104]:
df_daily_price['date'] = df_daily_price['date'].astype(str).str[:10]
df_daily_price['date'] = pd.to_datetime(df_daily_price['date'], format='%Y-%m-%d', errors='coerce')

In [105]:
len(df_daily_price.tic.unique())

861

In [115]:
len(fund_df.tic.unique())

924

In [114]:
fund_df['datadate'] = pd.to_datetime(fund_df['datadate'])
df_daily_price['date'] = pd.to_datetime(df_daily_price['date'])
fund_df = fund_df.merge(df_daily_price, left_on=['datadate', 'tic'], right_on=['date', 'tic'], how='left')
fund_df.drop(columns=['datadate'], inplace=True)

In [116]:
l_df = list(fund_df.groupby('tic'))
for tic,df in l_df:
    df.reset_index(inplace=True, drop=True)
    df.sort_values('date')
    # our goal is to predict next quarter's return
    df['y_return'] = np.log(df['adj_close_q'].shift(-1) / df['adj_close_q'])

In [117]:
fund_df = pd.concat([x[1] for x in l_df])

In [ ]:
# fund_df.dropna(subset = ['y_return'])

,gvkey,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,cusip_x,...,ret,bid,ask,shrout,cfacpr,cfacshr,openprc,numtrd,retx,y_return
3,126554,2001,2,10,INDL,C,D,STD,A,00846U101,...,0.005671,23.402630,24.252211,NaN,1.0,1.0,23.740053,NaN,0.005671,-0.310062
10,126554,2001,4,10,INDL,C,D,STD,A,00846U101,...,0.003153,13.346248,13.978915,NaN,1.0,1.0,13.496883,NaN,0.003153,0.309556
11,126554,2002,1,10,INDL,C,D,STD,A,00846U101,...,0.030910,17.570052,18.305150,NaN,1.0,1.0,17.654406,NaN,0.030910,-0.009934
12,126554,2002,2,10,INDL,C,D,STD,A,00846U101,...,0.013491,17.768890,18.407582,NaN,1.0,1.0,17.985803,NaN,0.013491,-0.464759
13,126554,2002,3,10,INDL,C,D,STD,A,00846U101,...,0.018888,10.755330,11.387997,NaN,1.0,1.0,11.237362,NaN,0.018888,-0.317064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,23809,2022,4,8,INDL,C,D,STD,AZO,053332102,...,-0.008334,2116.679932,2147.830078,NaN,1.0,1.0,2132.100098,NaN,-0.008334,0.196358
107,23809,2023,1,8,INDL,C,D,STD,AZO,053332102,...,0.008979,2536.139893,2587.600098,NaN,1.0,1.0,2559.129883,NaN,0.008979,-0.036510
108,23809,2023,2,8,INDL,C,D,STD,AZO,053332102,...,-0.033321,2483.540039,2584.959961,NaN,1.0,1.0,2584.959961,NaN,-0.033321,-0.040922
109,23809,2023,3,8,INDL,C,D,STD,AZO,053332102,...,-0.027621,2277.879883,2390.860107,NaN,1.0,1.0,2336.469971,NaN,-0.027621,0.058775


In [118]:
fund_df['pe'] = fund_df.prccq / fund_df.epspxq
fund_df['ps'] = fund_df.prccq / (fund_df.revtq/fund_df.cshoq)
fund_df['pb'] = fund_df.prccq / ((fund_df.atq-fund_df.ltq)/fund_df.cshoq)

In [119]:
items = [
    'date', # Date
    'gvkey',#gvkey unique identifier
    'tic', # Ticker
    'oiadpq', # Quarterly operating income
    'revtq', # Quartely revenue
    'niq', # Quartely net income
    'atq', # Total asset
    'teqq', # Shareholder's equity
    'epspiy', # EPS(Basic) incl. Extraordinary items
    'ceqq', # Common Equity
    'cshoq', # Common Shares Outstanding
    'dvpspq', # Dividends per share
    'actq', # Current assets
    'lctq', # Current liabilities
    'cheq', # Cash & Equivalent
    'rectq', # Recievalbles
    'cogsq', # Cost of  Goods Sold
    'invtq', # Inventories
    'apq',# Account payable
    'dlttq', # Long term debt
    'dlcq', # Debt in current liabilites
    'ltq', # Liabilities   
    'pe', #Price–earnings ratio
    'ps', #Price–sales ratio
    'pb', #Price-to-Book (P/B) Ratio
    'adj_close_q',#adjusted close price
    'y_return' #next quarter return
]

# Omit items that will not be used
fund_data = fund_df[items]


In [120]:
# Rename column names for the sake of readability
fund_data = fund_data.rename(columns={
    'oiadpq':'op_inc_q', # Quarterly operating income
    'revtq':'rev_q', # Quartely revenue
    'niq':'net_inc_q', # Quartely net income
    'atq':'tot_assets', # Assets
    'teqq':'sh_equity', # Shareholder's equity
    'epspiy':'eps_incl_ex', # EPS(Basic) incl. Extraordinary items
    'ceqq':'com_eq', # Common Equity
    'cshoq':'sh_outstanding', # Common Shares Outstanding
    'dvpspq':'div_per_sh', # Dividends per share
    'actq':'cur_assets', # Current assets
    'lctq':'cur_liabilities', # Current liabilities
    'cheq':'cash_eq', # Cash & Equivalent
    'rectq':'receivables', # Receivalbles
    'cogsq':'cogs_q', # Cost of  Goods Sold
    'invtq':'inventories', # Inventories
    'apq': 'payables',# Account payable
    'dlttq':'long_debt', # Long term debt
    'dlcq':'short_debt', # Debt in current liabilites
    'ltq':'tot_liabilities', # Liabilities   
})

In [121]:
# set data type to series
date = fund_data['date'].to_frame('date').reset_index(drop=True)
tic = fund_data['tic'].to_frame('tic').reset_index(drop=True)
gvkey = fund_data['gvkey'].to_frame('gvkey').reset_index(drop=True)
adj_close_q = fund_data['adj_close_q'].to_frame('adj_close_q').reset_index(drop=True)
y_return = fund_data['y_return'].to_frame('y_return').reset_index(drop=True)
# gsector = fund_data['gsector'].to_frame('gsector').reset_index(drop=True)
pe = fund_data['pe'].to_frame('pe').reset_index(drop=True)
ps = fund_data['ps'].to_frame('ps').reset_index(drop=True)
pb = fund_data['pb'].to_frame('pb').reset_index(drop=True)

In [122]:
# Calculate financial ratios

# Profitability ratios
# Operating Margin
OPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='OPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        OPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        OPM.iloc[i] = np.nan
    else:
        OPM.iloc[i] = np.sum(fund_data['op_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
OPM=pd.Series(OPM).to_frame().reset_index(drop=True)

# Net Profit Margin        
NPM = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='NPM')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        NPM[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        NPM.iloc[i] = np.nan
    else:
        NPM.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/np.sum(fund_data['rev_q'].iloc[i-3:i])
NPM=pd.Series(NPM).to_frame().reset_index(drop=True)

# Return On Assets
ROA = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROA')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROA[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROA.iloc[i] = np.nan
    else:
        ROA.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['tot_assets'].iloc[i]
ROA=pd.Series(ROA).to_frame().reset_index(drop=True)

# Return on Equity
ROE = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='ROE')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        ROE[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        ROE.iloc[i] = np.nan
    else:
        ROE.iloc[i] = np.sum(fund_data['net_inc_q'].iloc[i-3:i])/fund_data['sh_equity'].iloc[i]        
ROE=pd.Series(ROE).to_frame().reset_index(drop=True)

# For calculating valuation ratios in the next subpart, calculate per share items in advance
# Earnings Per Share       
EPS = fund_data['eps_incl_ex'].to_frame('EPS').reset_index(drop=True)

# Book Per Share
BPS = (fund_data['com_eq']/fund_data['sh_outstanding']).to_frame('BPS').reset_index(drop=True) # Need to check units

#Dividend Per Share
DPS = fund_data['div_per_sh'].to_frame('DPS').reset_index(drop=True)

# Liquidity ratios
# Current ratio
cur_ratio = (fund_data['cur_assets']/fund_data['cur_liabilities']).to_frame('cur_ratio').reset_index(drop=True)

# Quick ratio
quick_ratio = ((fund_data['cash_eq'] + fund_data['receivables'] )/fund_data['cur_liabilities']).to_frame('quick_ratio').reset_index(drop=True)

# Cash ratio
cash_ratio = (fund_data['cash_eq']/fund_data['cur_liabilities']).to_frame('cash_ratio').reset_index(drop=True)


# Efficiency ratios
# Inventory turnover ratio
inv_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='inv_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        inv_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        inv_turnover.iloc[i] = np.nan
    else:
        inv_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['inventories'].iloc[i]
inv_turnover=pd.Series(inv_turnover).to_frame().reset_index(drop=True)

# Receivables turnover ratio       
acc_rec_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_rec_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_rec_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_rec_turnover.iloc[i] = np.nan
    else:
        acc_rec_turnover.iloc[i] = np.sum(fund_data['rev_q'].iloc[i-3:i])/fund_data['receivables'].iloc[i]
acc_rec_turnover=pd.Series(acc_rec_turnover).to_frame().reset_index(drop=True)

# Payable turnover ratio
acc_pay_turnover = pd.Series(np.empty(fund_data.shape[0],dtype=object),name='acc_pay_turnover')
for i in range(0, fund_data.shape[0]):
    if i-3 < 0:
        acc_pay_turnover[i] = np.nan
    elif fund_data.iloc[i,1] != fund_data.iloc[i-3,1]:
        acc_pay_turnover.iloc[i] = np.nan
    else:
        acc_pay_turnover.iloc[i] = np.sum(fund_data['cogs_q'].iloc[i-3:i])/fund_data['payables'].iloc[i]
acc_pay_turnover=pd.Series(acc_pay_turnover).to_frame().reset_index(drop=True)

## Leverage financial ratios
# Debt ratio
debt_ratio = (fund_data['tot_liabilities']/fund_data['tot_assets']).to_frame('debt_ratio').reset_index(drop=True)

# Debt to Equity ratio
debt_to_equity = (fund_data['tot_liabilities']/fund_data['sh_equity']).to_frame('debt_to_equity').reset_index(drop=True)

In [123]:
# Create a dataframe that merges all the ratios
ratios = pd.concat([date,gvkey,tic,adj_close_q,y_return,OPM,NPM,ROA,ROE,EPS,BPS,DPS,
                    cur_ratio,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,
                   debt_ratio,debt_to_equity,pe,ps,pb], axis=1).reset_index(drop=True)

In [124]:
# Replace NAs infinite values with zero
final_ratios = ratios.copy()
final_ratios = final_ratios.fillna(0)
final_ratios = final_ratios.replace(np.inf,0)

In [125]:
final_ratios.to_csv('final_ratios_raw.csv')

In [126]:
final_ratios.head()

,date,gvkey,tic,adj_close_q,y_return,OPM,NPM,ROA,ROE,EPS,...,quick_ratio,cash_ratio,inv_turnover,acc_rec_turnover,acc_pay_turnover,debt_ratio,debt_to_equity,pe,ps,pb
0,0,10507,0033A,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.56,...,0.169339,0.014917,0.000000,0.000000,0.000000,0.695946,0.0,73.883929,6.383067,1.430224
1,0,10507,0033A,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.46,...,0.271284,0.010014,0.000000,0.000000,0.000000,0.696722,0.0,47.500000,5.677115,1.461822
2,0,10507,0033A,0.0,0.0,0.000000,0.000000,0.000000,0.0,3.05,...,0.266745,0.020813,0.000000,0.000000,0.000000,0.685703,0.0,25.000000,4.625658,1.314168
3,0,10507,0033A,0.0,0.0,0.333037,0.143130,0.034051,0.0,3.35,...,0.207633,0.010034,9.085628,15.509621,8.705043,0.684889,0.0,135.833333,6.244855,1.358811
4,0,10507,0033A,0.0,0.0,0.312177,0.131014,0.030764,0.0,0.51,...,0.313543,0.146420,9.658611,16.798462,11.588044,0.709488,0.0,67.156863,5.149707,1.222281


In [127]:
features_column_financial=[ 'OPM', 'NPM', 'ROA', 'ROE', 'EPS', 'BPS', 'DPS', 'cur_ratio',
       'quick_ratio', 'cash_ratio', 'inv_turnover', 'acc_rec_turnover',
       'acc_pay_turnover', 'debt_ratio', 'debt_to_equity', 'pe', 'ps', 'pb']

In [128]:
def handle_nan(df,features_column_financial):
    ##handle nan, inf
    df=df.drop(list(df[df.adj_close_q==0].index)).reset_index(drop=True)
    df['y_return'] = pd.to_numeric(df['y_return'], errors='coerce')
    for col in features_column_financial:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['y_return'].replace([np.nan,np.inf,-np.inf], np.nan, inplace=True)
    df[features_column_financial].replace([np.nan, np.inf, -np.inf], np.nan, inplace=True)
    dropped_col = []
    for col in features_column_financial:
        if np.any(~np.isfinite(df[col])):
            df.drop(columns=[col], axis=1, inplace=True)
    df.dropna(axis=0, inplace=True)
    df=df.reset_index(drop=True)
    print("dropped_col: ",dropped_col)
    return df

In [129]:
final_ratios=handle_nan(final_ratios,features_column_financial)

dropped_col:  []


In [130]:
final_ratios.date=final_ratios.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [131]:
final_ratios.to_csv('final_ratios.csv')